# Part 1

In [1]:
# Stolen from day 6 and modified for today.
import itertools

class Matrix:
    ''' A dense matrix. '''
    def __init__(self, rows, cols, initial=' '):
        self._rows = rows
        self._cols = cols
        self._cells = [[initial] * cols for _ in range(rows)]
    
    def __repr__(self):
        return 'Matrix<{}x{}>'.format(self._rows, self._cols)
    
    def __str__(self):
        render = ''
        for row in self._cells:
            render += ''.join(str(v) for v in row) + '\n'
        return render

    def __getitem__(self, key):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        return self._cells[row][col]
    
    def __setitem__(self, key, val):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        self._cells[row][col] = val
    
    def clone(self):
        m = Matrix(self._rows, self._cols)
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            m[row,col] = self[row,col]
        return m

    def get(self, row, col, default=None):
        try:
            return self[row,col]
        except (IndexError,KeyError):
            return default
    
    @property
    def size(self):
        return self._rows, self._cols

    def values(self):
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield self[row, col]

In [2]:
class Player:
    id_gen = itertools.count()
    
    def __init__(self, power=None):
        self._id = next(Player.id_gen)
        self._attack = power or 3
        self._hp = 200
    
    @property
    def hp(self):
        return self._hp
    
    def attack(self, other):
        other._hp -= self._attack

    def __repr__(self):
        return '<{} id={} hp={}>'.format(self.__class__.__name__, self._id, self._hp)
    
class Goblin(Player):
    def __str__(self):
        return 'G'

class Elf(Player):
    def __str__(self):
        return 'E'

In [3]:
def parse_map(text, elf_power=None):
    ''' Parse text representation of map and return a matrix. '''
    lines = text.split('\n')
    rows = len(lines)
    cols = len(lines[0])
    mat = Matrix(rows, cols)
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char == 'E':
                mat[row,col] = Elf(elf_power)
            elif char == 'G':
                mat[row,col] = Goblin()
            else:
                mat[row,col] = char
    return mat

In [4]:
import collections

def print_map(mat):
    ''' Display the map with player information in the margin. '''
    rows, cols = mat.size
    for r in range(rows):
        players = ''
        row = ''
        for c in range(cols):
            cell = mat[r,c]
            if isinstance(cell, Player):
                players += repr(cell) + ' '
            row += str(cell)
        print('{} {}'.format(row, players))

In [5]:
test_text = '''#######
#E..G.#
#...#.#
#.G.#G#
#######'''
test_mat = parse_map(test_text)
print_map(test_mat)

####### 
#E..G.# <Elf id=0 hp=200> <Goblin id=1 hp=200> 
#...#.# 
#.G.#G# <Goblin id=2 hp=200> <Goblin id=3 hp=200> 
####### 


In [6]:
def find_adjacent(mat, row, col, cls):
    ''' Search the cells adjacent to (row,col) for a player of type cls. 
    If multiple adjacent, return the one with the fewest HP. '''
    opponents = [
        mat.get(row-1, col  ),
        mat.get(row  , col-1),
        mat.get(row  , col+1),
        mat.get(row+1, col  ),
    ]
    opponents_hp = [(opp.hp, opp) for opp in opponents if isinstance(opp, cls)]
    opponents_hp.sort(key=lambda t: t[0])
    try:
        return opponents_hp[0][1]
    except IndexError:
        return None

In [7]:
find_adjacent(test_mat, 1, 2, Goblin)

In [8]:
find_adjacent(test_mat, 1, 2, Elf)

<Elf id=0 hp=200>

In [9]:
def find_remote(mat, row, col, cls, debug=False):
    ''' Search the map for the player of type cls closest to (row,col). 
    If multiple cells found, return the first one in "reading order". 
    Returns the new_row, new_col that the player should move to. Returns
    None if no players can be reached. '''
    # Make a map to store the distances from the initial cell.
    dist_mat = mat.clone()
    dist_mat[row, col] = '.'
    stack = [((row,col),)]
    while stack:
        path = stack.pop(0)
        r, c = path[0]
        cell = dist_mat.get(r, c)
        if cell == '.':
            dist_mat[r,c] = len(path) - 1
            stack += [
                ((r-1, c  ), *path),
                ((r  , c-1), *path),
                ((r  , c+1), *path),
                ((r+1, c  ), *path),
            ]
        elif isinstance(cell, cls):
            break
    else:
        if debug:
            print(dist_mat)
        return None
    if debug:
        print(dist_mat)
    return path[-2]

In [10]:
test_text = '''#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########'''
test_mat = parse_map(test_text)
find_remote(test_mat, 1, 4, Elf, debug=True)

#########
#G21012G#
#.32123.#
#..323..#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########



(2, 4)

In [11]:
find_remote(test_mat, 1, 7, Elf, debug=True)

#########
#G.6G210#
#.654321#
#..65432#
#G..E54G#
#.....5.#
#.......#
#G..G..G#
#########



(1, 6)

In [12]:
class GameOver(Exception):
    pass

def play_round(mat):
    ''' Play one round of the game, updating mat in place. 
    Raises GameOver if the game is done. '''
    # Determine order of play
    players = list()
    locations = dict()
    rows, cols = mat.size
    for r, c in itertools.product(range(rows), range(cols)):
        cell = mat[r,c]
        if isinstance(cell, Player):
            locations[cell] = (r,c)
            players.append(cell)

    for player in players:
        # If the player died, then skip this turn.
        if player.hp <= 0:
            continue

        if ((isinstance(player, Elf) and
             len([cell for cell in mat.values() if isinstance(cell, Goblin)])==0) or
            (isinstance(player, Goblin) and 
             len([cell for cell in mat.values() if isinstance(cell, Elf)])==0)):
            raise GameOver()

        # Does the player need to move?
        row, col = locations[player]
        opp_class = Goblin if isinstance(player, Elf) else Elf
        opp = find_adjacent(mat, row, col, opp_class)
        if opp is None:
            # Move towards opponent
            move = find_remote(mat, row, col, opp_class)
            if move is None:
                continue
            mat[row, col] = '.'
            row, col = move
            mat[row, col] = player
            locations[player] = row, col
        
        # Can the player attack?
        opp = find_adjacent(mat, row, col, Goblin if isinstance(player, Elf) else Elf)
        if opp is not None:
            # Do attack
            player.attack(opp)
            if opp.hp <= 0:
                opp_row, opp_col = locations[opp]
                mat[opp_row,opp_col] = '.'

In [13]:
test_text = '''#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########'''
test_mat = parse_map(test_text)
print_map(test_mat)

######### 
#G..G..G# <Goblin id=13 hp=200> <Goblin id=14 hp=200> <Goblin id=15 hp=200> 
#.......# 
#.......# 
#G..E..G# <Goblin id=16 hp=200> <Elf id=17 hp=200> <Goblin id=18 hp=200> 
#.......# 
#.......# 
#G..G..G# <Goblin id=19 hp=200> <Goblin id=20 hp=200> <Goblin id=21 hp=200> 
######### 


In [16]:
# Run this cell multiple times.
play_round(test_mat)
print_map(test_mat)

######### 
#.......# 
#..GGG..# <Goblin id=13 hp=200> <Goblin id=14 hp=191> <Goblin id=15 hp=200> 
#..GEG..# <Goblin id=16 hp=200> <Elf id=17 hp=185> <Goblin id=18 hp=200> 
#G..G...# <Goblin id=19 hp=200> <Goblin id=20 hp=200> 
#......G# <Goblin id=21 hp=200> 
#.......# 
#.......# 
######### 


In [17]:
test_text = '''#######   
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######'''
test_mat = parse_map(test_text)
print_map(test_mat)
rounds = 0

#######    
#.G...#    <Goblin id=22 hp=200> 
#...EG#    <Elf id=23 hp=200> <Goblin id=24 hp=200> 
#.#.#G#    <Goblin id=25 hp=200> 
#..G#E#    <Goblin id=26 hp=200> <Elf id=27 hp=200> 
#.....#    
#######    


In [65]:
# Run this cell multiple times.
try:
    play_round(test_mat)
except GameOver:
    print('Game Over!')
print_map(test_mat)
rounds += 1 
print(rounds)

Game Over!
#######    
#G....#    <Goblin id=22 hp=200> 
#.G...#    <Goblin id=24 hp=131> 
#.#.#G#    <Goblin id=25 hp=59> 
#...#.#    
#....G#    <Goblin id=26 hp=200> 
#######    
48


In [66]:
def sum_hp(mat):
    return sum(p.hp for p in mat.values() if isinstance(p, Player))

In [67]:
sum_hp(test_mat)

590

In [68]:
47 * 590

27730

In [72]:
def play_all(mat):
    round = 0
    try:
        while True:
            play_round(mat)
            round += 1
    except:
        pass
    elves = len([cell for cell in mat.values() if isinstance(cell, Elf)])
    goblins = len([cell for cell in mat.values() if isinstance(cell, Goblin)])
    sum_ = sum_hp(mat)
    print('elves={} goblins={} round={} sum={} total={}'.format(
        elves, goblins, round, sum_, round * sum_))

In [73]:
test_text = '''#######   
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######'''
test_mat = parse_map(test_text)
play_all(test_mat)

elves=0 goblins=4 round=47 sum=590 total=27730


In [74]:
test_text = '''#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######'''
test_mat = parse_map(test_text)
play_all(test_mat)

elves=5 goblins=0 round=37 sum=982 total=36334


In [75]:
test_text = '''#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######'''
test_mat = parse_map(test_text)
play_all(test_mat) 

elves=5 goblins=0 round=46 sum=859 total=39514


In [76]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text)
print(mat)

################################
##########################..####
##########################...###
####################G..#......##
##############.#####G....G....##
##############...#####..##..#.##
################..##...#########
##############....#....#########
#########..###G........E######.#
#########........#..GG..####...#
#########......................#
#########..........G..G........#
########...G..#####............#
#######......#######........####
#####.G.....#########E.......###
######......#########........###
#####.......#########....E######
#####.G...G.#########.....######
######..G...#########.....######
#####...G....#######.......#####
###....G......#####.......######
#.#G.....E......E.........#..###
#............G..G.G.#.#...E....#
####.....................#####.#
########...........EE.##.#######
########..............##########
#########.....G.....E.##########
#########............###########
##########..........############
##########.......E.....#########
##########

In [77]:
play_all(mat)

elves=0 goblins=16 round=69 sum=2804 total=193476


# Part 2

How many elves do we start with?

In [78]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text)
elves = len([cell for cell in mat.values() if isinstance(cell, Elf)])
print(elves)

10


In [80]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=100)
play_all(mat)

elves=10 goblins=0 round=14 sum=1865 total=26110


In [81]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=50)
play_all(mat)

elves=10 goblins=0 round=20 sum=1667 total=33340


In [82]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=25)
play_all(mat)

elves=9 goblins=0 round=27 sum=1401 total=37827


In [83]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=32)
play_all(mat)

elves=9 goblins=0 round=25 sum=1479 total=36975


In [84]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=44)
play_all(mat)

elves=10 goblins=0 round=25 sum=1589 total=39725


In [85]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=38)
play_all(mat)

elves=10 goblins=0 round=24 sum=1532 total=36768


In [86]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=35)
play_all(mat)

elves=10 goblins=0 round=24 sum=1532 total=36768


In [87]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=34)
play_all(mat)

elves=10 goblins=0 round=24 sum=1532 total=36768


In [88]:
with open('input.txt') as input_:
    text = input_.read()
mat = parse_map(text, elf_power=33)
play_all(mat)

elves=9 goblins=0 round=25 sum=1479 total=36975
